In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB as MNB
import numpy as np
import pandas as pd
import csv
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from nltk.tokenize import TweetTokenizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

In [2]:
tokenize = TweetTokenizer().tokenize

In [3]:
train = pd.read_csv("train.tsv", sep="\t")
test = pd.read_csv("test.tsv", sep="\t")
sub = pd.read_csv('sampleSubmission.csv', sep=",")

In [4]:
print(train.shape)
print(list(train))

(156060, 4)
['PhraseId', 'SentenceId', 'Phrase', 'Sentiment']


In [5]:
train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [6]:
test.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [7]:
categories = train['Sentiment'].unique()

In [8]:
X = TfidfVectorizer().fit_transform(t for t in train['Phrase'])

In [9]:
count_vector = CountVectorizer()
train_count = count_vector.fit_transform(train['Phrase'])

In [10]:
print(train_count.shape)

(156060, 15240)


In [11]:
print(train_count[0])

  (0, 12857)	1
  (0, 8807)	1
  (0, 13681)	1
  (0, 593)	1
  (0, 9085)	1
  (0, 1879)	1
  (0, 602)	1
  (0, 9204)	1
  (0, 14888)	2
  (0, 12424)	1
  (0, 5595)	1
  (0, 529)	1
  (0, 5837)	1
  (0, 5323)	2
  (0, 5821)	2
  (0, 7217)	2
  (0, 14871)	1
  (0, 13503)	1
  (0, 288)	1
  (0, 13505)	3
  (0, 3490)	1
  (0, 4577)	1
  (0, 9227)	4
  (0, 11837)	1


In [12]:
count_vector.vocabulary_.get(u'movie')

8791

In [13]:
print(count_vector)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


In [14]:
count_vector.get_feature_names()

['000',
 '10',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '10th',
 '11',
 '110',
 '112',
 '12',
 '120',
 '127',
 '129',
 '12th',
 '13',
 '13th',
 '14',
 '140',
 '146',
 '15',
 '15th',
 '16',
 '163',
 '168',
 '170',
 '1790',
 '18',
 '1899',
 '19',
 '1915',
 '1920',
 '1930s',
 '1933',
 '1937',
 '1938',
 '1940s',
 '1950',
 '1950s',
 '1952',
 '1953',
 '1957',
 '1958',
 '1959',
 '1960',
 '1960s',
 '1962',
 '1970',
 '1970s',
 '1971',
 '1972',
 '1973',
 '1975',
 '1979',
 '1980',
 '1980s',
 '1984',
 '1986',
 '1987',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1997',
 '1998',
 '1999',
 '19th',
 '20',
 '2000',
 '2001',
 '2002',
 '20th',
 '21',
 '21st',
 '22',
 '24',
 '2455',
 '25',
 '26',
 '270',
 '295',
 '30',
 '300',
 '3000',
 '30s',
 '37',
 '3d',
 '40',
 '40s',
 '42',
 '451',
 '48',
 '4ever',
 '4th',
 '4w',
 '50',
 '500',
 '50s',
 '51',
 '51st',
 '52',
 '53',
 '5ths',
 '60',
 '60s',
 '65',
 '65th',
 '66',
 '70',
 '70s',
 '71',
 '72',
 '75',
 '77',
 '78',
 '7th',
 '8

In [15]:
tf_transformer = TfidfTransformer(use_idf=False).fit(train_count)

In [16]:
train_tf = tf_transformer.transform(train_count)

In [17]:
tfidf_transformer = TfidfTransformer()

In [18]:
train_tfidf = tfidf_transformer.fit_transform(train_count)

In [19]:
CV = 2
models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MNB(),
    LogisticRegression(random_state=0),
]

for model in models:
    accuracy = cross_val_score(model,train_tfidf,train['Sentiment'],\
                            scoring='accuracy',cv=CV)
    print(str(model),":",accuracy)
    print("\n\n")

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False) : [0.50993836 0.50995143]



LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0) : [0.58038472 0.57417114]



MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True) : [0.55471543 0.55223058]



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          ve

In [20]:
classf  =  MNB().fit(train_tfidf , train['Sentiment'])
LSVC = LinearSVC().fit(train_tfidf, train['Sentiment'])

In [21]:
test_counts = count_vector.transform(test['Phrase'])

In [22]:
test_tfidf = tfidf_transformer.transform(test_counts)
#X = TfidfVectorizer().fit_transform(t for t in X_train['Phrase'])

In [23]:
predicted = classf.predict(test_tfidf)
predicted_LSVC = LSVC.predict(test_tfidf)

In [24]:
with open('Sentiment_Naive_Bayes2.csv', 'w') as csvfile:
    csvfile.write('PhraseId,Sentiment\n')
    for i,j in zip(test['PhraseId'],predicted):
        csvfile.write('{}, {}\n'.format(i, j))

In [25]:
with open('Sentiment_LSVC.csv', 'w') as csvfile:
    csvfile.write('PhraseId,Sentiment\n')
    for i,j in zip(test['PhraseId'],predicted_LSVC):
        csvfile.write('{}, {}\n'.format(i, j))
#Score 0.61

In [26]:
def arraytotext(records):
    return [" ".join(record).lower() for record in records]

In [27]:
tfidf = TfidfVectorizer(binary=True)

In [28]:
[t for t in train['Phrase']]

['A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .',
 'A series of escapades demonstrating the adage that what is good for the goose',
 'A series',
 'A',
 'series',
 'of escapades demonstrating the adage that what is good for the goose',
 'of',
 'escapades demonstrating the adage that what is good for the goose',
 'escapades',
 'demonstrating the adage that what is good for the goose',
 'demonstrating the adage',
 'demonstrating',
 'the adage',
 'the',
 'adage',
 'that what is good for the goose',
 'that',
 'what is good for the goose',
 'what',
 'is good for the goose',
 'is',
 'good for the goose',
 'good',
 'for the goose',
 'for',
 'the goose',
 'goose',
 'is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .',
 'is also good for the gander , some of which occasionally amuses but none of 

In [29]:
print(tokenize(train['Phrase'][0]))

['A', 'series', 'of', 'escapades', 'demonstrating', 'the', 'adage', 'that', 'what', 'is', 'good', 'for', 'the', 'goose', 'is', 'also', 'good', 'for', 'the', 'gander', ',', 'some', 'of', 'which', 'occasionally', 'amuses', 'but', 'none', 'of', 'which', 'amounts', 'to', 'much', 'of', 'a', 'story', '.']


In [56]:
y = train['Sentiment']

In [43]:
tfidf = TfidfVectorizer(ngram_range=(1, 2), tokenizer=tokenize)
full_text = list(train['Phrase'].values) + list(test['Phrase'].values)
tfidf.fit(full_text)


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f2d50928208>>,
        use_idf=True, vocabulary=None)

In [31]:
train_features = tfidf.transform((train['Phrase']))
test_features = tfidf.transform((test['Phrase']))

In [32]:
classifier = LogisticRegression(random_state=0)
model = OneVsRestClassifier(classifier)

In [33]:
%%time
model.fit(train_features, train['Sentiment'])

CPU times: user 9.34 s, sys: 108 ms, total: 9.45 s
Wall time: 4.78 s


OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=1)

In [34]:
scores = cross_val_score(model, train_features, train['Sentiment'],\
                         scoring='accuracy', n_jobs=-1, cv=3)
print('Cross-validation mean accuracy {0:.2f}%, std {1:.2f}.'.format(np.mean(scores) * 100, np.std(scores) * 100))

Cross-validation mean accuracy 56.55%, std 0.07.


In [35]:
predicted_Logis = model.predict(test_features)

In [36]:
sub['Sentiment'] = predicted_Logis
sub.to_csv("blend.csv", index=False)

In [37]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GRU, CuDNNGRU, CuDNNLSTM, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping

/home/keanu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [38]:
tk = Tokenizer(lower = True, filters='')
tk.fit_on_texts(full_text)

In [39]:
train_tokenized = tk.texts_to_sequences(train['Phrase'])
test_tokenized = tk.texts_to_sequences(test['Phrase'])

In [40]:
max_len = 50
X_train = pad_sequences(train_tokenized, maxlen = max_len)
X_test = pad_sequences(test_tokenized, maxlen = max_len)

In [52]:
embedding_path = "../crawl-300d-2M.vec"

In [53]:
embed_size = 300
max_features = 20000

In [54]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

word_index = tk.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words + 1, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [58]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
y_ohe = ohe.fit_transform(y.values.reshape(-1, 1))

In [59]:
file_path = "best_model.hdf5"
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                              save_best_only = True, mode = "min")
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 3)

def build_model(lr = 0.0, lr_d = 0.0, units = 0, dr = 0.0):
    inp = Input(shape = (max_len,))
    x = Embedding(19479, embed_size, weights = [embedding_matrix], trainable = False)(inp)
    x1 = SpatialDropout1D(dr)(x)

    x_gru = Bidirectional(CuDNNGRU(units, return_sequences = True))(x1)
    x1 = Conv1D(32, kernel_size=3, padding='valid', kernel_initializer='he_uniform')(x_gru)
    avg_pool1_gru = GlobalAveragePooling1D()(x1)
    max_pool1_gru = GlobalMaxPooling1D()(x1)
    
    x3 = Conv1D(32, kernel_size=2, padding='valid', kernel_initializer='he_uniform')(x_gru)
    avg_pool3_gru = GlobalAveragePooling1D()(x3)
    max_pool3_gru = GlobalMaxPooling1D()(x3)
    
    x_lstm = Bidirectional(CuDNNLSTM(units, return_sequences = True))(x1)
    x1 = Conv1D(32, kernel_size=3, padding='valid', kernel_initializer='he_uniform')(x_lstm)
    avg_pool1_lstm = GlobalAveragePooling1D()(x1)
    max_pool1_lstm = GlobalMaxPooling1D()(x1)
    
    x3 = Conv1D(32, kernel_size=2, padding='valid', kernel_initializer='he_uniform')(x_lstm)
    avg_pool3_lstm = GlobalAveragePooling1D()(x3)
    max_pool3_lstm = GlobalMaxPooling1D()(x3)
    
    
    x = concatenate([avg_pool1_gru, max_pool1_gru, avg_pool3_gru, max_pool3_gru,
                    avg_pool1_lstm, max_pool1_lstm, avg_pool3_lstm, max_pool3_lstm])
    x = BatchNormalization()(x)
    x = Dropout(0.1)(Dense(128,activation='relu') (x))
    x = BatchNormalization()(x)
    x = Dropout(0.1)(Dense(64,activation='relu') (x))
    x = Dense(5, activation = "sigmoid")(x)
    model = Model(inputs = inp, outputs = x)
    model.compile(loss = "binary_crossentropy", optimizer = Adam(lr = lr, decay = lr_d), metrics = ["accuracy"])
    history = model.fit(X_train, y_ohe, batch_size = 128, epochs = 1, validation_split=0.1, 
                        verbose = 1, callbacks = [check_point, early_stop])
    model = load_model(file_path)
    return model

In [ ]:
model = build_model(lr = 1e-3, lr_d = 0, units = 128, dr = 0.5)
#Can't run on my computer since I don't have a GPU for tensorflow
#Used Kaggle instead to run this program

In [ ]:
y_pred2=model2.predict_classes(X_test, verbose=1)

In [ ]:
sub.Sentiment=y_pred2
sub.to_csv('sub2.csv',index=False)
sub.head()